$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\renewcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal {#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\abs}[1]{\left\lvert#1\right\rvert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\Tr}[0]{^\top}
\newcommand{\grad}[0]{\nabla}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\Var}[1]{\mathrm{Var}\left[#1\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[0]{\middle\vert}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\DeclareMathOperator*{\trace}{trace}
\newcommand{\1}[1]{\mathbb{I}\left\{#1\right\}}
\newcommand{\setof}[1]{\left\{#1\right\}}
\newcommand{\DO}[1]{\mathrm{do}\left(#1\right)}
\newcommand{\indep}{\perp \!\!\! \perp}
$$


# <center>Causal Inference 097400, Winter 2019-20<br><br>Final Project</center>

#### <center>Aviv Rosenberg<br>`avivr@cs`</center>

##### <center>April, 2020<br></center>


In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd

DATA_FILE = 'data/nsr_chf_af_ag-y45o60-winfilt2.xlsx'
OUT_DIR = 'out/'

os.makedirs(OUT_DIR, exist_ok=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from ciproj import data

dfs = data.load_mhrv_xls(DATA_FILE, sheet_names=['NSR', 'AGING'])

assert not np.any(np.isnan(dfs['NSR'].values))
assert not np.any(np.isnan(dfs['AGING'].values))

Loaded NSR: 3156 samples, 58 features
Loaded AGING: 5809 samples, 58 features


In [4]:
df_full = data.create_ci_dataset(df_control=dfs['NSR'], df_treated=dfs['AGING'])

AttributeError: 'Index' object has no attribute 'levels'

In [27]:
data.create_multiscale_outcomes(df_full)

,RR,NN,AVNN,SDNN,RMSSD,pNN50,SEM,BETA,HF_NORM,HF_PEAK,...,alpha1,alpha2,SampEn,PIP,IALS,PSS,PAS,T,y_mse_int,y_mse_slope
Row,,,,,,,,,,,,,,,,,,,,,
nsr047_1,872,869,661.059753,47.829563,19.061163,1.843318,1.622507,0.126201,9.502737,0.304977,...,1.480327,0.937134,0.941021,34.407364,0.345622,26.006905,8.745685,0,1.108570,0.095309
nsr047_2,872,869,674.185486,38.503155,17.583624,1.036866,1.306130,-0.127805,6.397126,0.365523,...,1.366680,1.168168,2.037776,38.089760,0.382488,29.228998,9.666283,0,1.908045,-0.050479
nsr047_3,857,857,699.706482,55.378662,19.560434,1.285047,1.891699,-0.604867,8.341562,0.159216,...,1.410178,1.121445,0.958004,36.056011,0.362150,26.954493,7.117853,0,1.140616,0.045506
nsr047_4,929,928,643.167419,79.910538,19.377684,1.510248,2.623192,-0.604719,5.665492,0.353564,...,1.419032,1.038669,0.521856,33.728447,0.338727,24.245689,6.681035,0,0.630028,0.048994
nsr047_5,929,929,645.460510,71.772980,19.338352,1.508621,2.354795,-1.708907,4.419215,0.372251,...,1.587830,0.729617,0.955613,36.813778,0.369612,28.417654,8.934338,0,0.871707,0.081506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nsr046_134,677,673,869.172546,75.834610,60.977371,5.059524,2.923209,-0.423178,13.006993,0.287784,...,0.767257,0.927866,1.060477,51.411591,0.516369,48.737000,18.127787,1,0.846738,0.020507
nsr046_135,806,803,732.215149,109.252686,65.208443,3.491272,3.855444,-1.204573,3.312317,0.335624,...,0.886307,0.891142,0.504730,50.311333,0.504988,45.828144,26.151930,1,0.335618,0.057916
nsr046_136,739,732,729.988464,88.629013,50.925941,3.009576,3.275823,-0.072055,20.510777,0.246672,...,0.637720,0.928644,0.520563,50.819672,0.510260,47.131149,24.316940,1,0.495591,0.028389
